In [1]:
import glob
print(glob.glob("/share/*"))

['/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1']


In [2]:
import sys
import os

import torch
from torch import nn as nn
import glob

from t4.timeseries_transformer3 import TransformerConfig, TransformerRunner

from timeseries.csv_reader import read_and_merge_csv_files

from t3_karpathy.commons.commons import BaseTransformerConfig, min_max_scaling, extract_sliding_windows, z_scale, diff
from t3_karpathy.token_codec import TokenCodec
from t4.generic_dataloader import GenericDataloader
from timeseries.timeseries_transformer import TimeseriesDataloader

import torch
import matplotlib.pyplot as plt

In [3]:
stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]

directory_path = '/share/us-stock-dataset/Data/Stocks'

# stocks_to_load = [s.split("\\")[1].replace(".csv", "") for s in glob.glob(directory_path + "//*.csv")]
#
# stock_number_to_load = 200
#
# stocks_to_load = stocks_to_load[1000:1000+stock_number_to_load]

dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

print(dataloader.get_data().shape)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

In [4]:
data_loader_data = dataloader.get_data().transpose(0, 1)
print(data_loader_data.shape)

torch.Size([145, 5283])


In [5]:
data_loader_data

tensor([[  0.8495,   0.7778,   0.7892,  ..., 134.6764, 132.8832, 131.7501],
        [  1.5927,   1.5927,   1.5927,  ..., 221.2300, 221.9967, 231.5933],
        [ 43.9297,  40.5740,  38.0572,  ..., 116.0503, 115.4594, 115.6170],
        ...,
        [ 19.9000,  19.9000,  19.9000,  ...,  19.9000,  19.9000,  19.9000],
        [ 15.6074,  15.6074,  15.6074,  ...,  14.4057,  15.1121,  14.7217],
        [  9.8600,   9.8600,   9.8600,  ...,   9.8600,   9.8600,   9.8600]],
       device='cuda:0')

In [6]:
diff_data = diff(data_loader_data, 1)
data = z_scale(diff_data, 1)

In [7]:
diff_data[0]

tensor([-0.0716,  0.0114, -0.0683,  ...,  4.6505, -1.7932, -1.1331],
       device='cuda:0')

In [8]:
torch.std(data[1])

tensor(1., device='cuda:0')

In [9]:
windowed = extract_sliding_windows(data, 8)
windowed.shape

torch.Size([145, 5275, 8])

In [10]:
windowed[0, 1, :]

tensor([-0.0209, -0.1449,  0.0146, -0.0593, -0.0977, -0.1043,  0.0744,  0.0050],
       device='cuda:0')

In [11]:
data[0,:]

tensor([-0.1501, -0.0209, -0.1449,  ...,  7.2019, -2.8304, -1.8027],
       device='cuda:0')

In [12]:
flat_windowed = windowed.reshape(windowed.shape[0]* windowed.shape[1], -1)
print(flat_windowed.shape)

torch.Size([764875, 8])


In [ ]:
device = 'cuda'

torch.manual_seed(1337)

config = TransformerConfig(
    input_embed=flat_windowed.shape[1], 
    precision=torch.bfloat16, 
    batch_size=128, 
    block_size=1, 
    n_embed=64, 
    n_head=4, 
    n_layer=4, 
    learning_rate=0.001
)

runner = TransformerRunner(config, flat_windowed)

runner.train_iterate_n(25000)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(dataloader.token_codec.decode(runner.generate(context, max_new_tokens=2000)[0].tolist()))

0.487448 M parameters
step 0: train loss 1.0102, val loss 0.7813, time/iter 2.86102294921875e-08
saved model version 1
step 100: train loss 0.4173, val loss 0.3308, time/iter 0.0489008378982544
step 200: train loss 0.3985, val loss 0.3242, time/iter 0.04117295026779175
step 300: train loss 0.3992, val loss 0.3150, time/iter 0.040119249820709225
step 400: train loss 0.3907, val loss 0.2801, time/iter 0.039424393177032474
step 500: train loss 0.3842, val loss 0.3014, time/iter 0.04054482936859131
step 600: train loss 0.3843, val loss 0.3089, time/iter 0.03979179859161377
step 700: train loss 0.3972, val loss 0.2947, time/iter 0.0401300048828125
step 800: train loss 0.3692, val loss 0.2837, time/iter 0.036434953212738035
step 900: train loss 0.3867, val loss 0.3073, time/iter 0.04791674852371216
step 1000: train loss 0.3626, val loss 0.3106, time/iter 0.04624037265777588
saved model version 2
step 1100: train loss 0.3860, val loss 0.3057, time/iter 0.044865119457244876
step 1200: train lo

In [ ]:

# Generate a sample tensor (you can replace this with your tensor)
tensor = data_loader_data.to('cpu')

# Plotting
plt.figure(figsize=(15, 7))
for i in range(tensor.shape[0]):
    plt.plot(tensor[i].numpy(), linewidth=0.5)  # Convert tensor to numpy for plotting

plt.title("145 Time Series")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.show()


In [ ]:
import pandas as pd
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

import plotly.express as px

# Sample data
data = {
    'Date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
    'Value': [1, 3, 5, 6, 8, 10, 12, 14, 16, 18]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Plotting using Plotly
fig = px.line(df, x='Date', y='Value', title='Time Series')
fig.show()


In [ ]:
!pip install plotly